In [ ]:
import os
import csv
import cantera as ct
import numpy as np
import sdtoolbox.postshock as sdps
import sdtoolbox.reflections as sdrf

mech = 'gri30.cti'
gas = ct.Solution(mech)
gas1s = ct.Solution(mech)
gas2s = ct.Solution(mech)

print('set initial parameters of mixture:')
print('input mole fractions of fuel')
h2fr = float(input('H2 fraction = '))
ch4fr = float(input('CH4 fraction = '))
c2h6fr = float(input('C2H6 fraction = '))
c3h8fr = float(input('C3H8 fraction = '))
eq_ratio = float(input('equivalence ratio = '))
temp = float(input('T (K)= '))
press = float(input('pressure (Pa, more than 101425)= '))
oxid = int(input('oxidizer: air - 1, or oxygen - 2?'))

gas.TPX = temp, press, ('O2:1,N2:3.76')
if(oxid == 1.0): 
    gas.set_equivalence_ratio(eq_ratio, 'H2:%s,CH4:%s,C2H6:%s,C3H8:%s'%(h2fr,ch4fr,c2h6fr,c3h8fr), 'O2:1.0,N2:3.76')
else: 
    gas.set_equivalence_ratio(eq_ratio, 'H2:%s,CH4:%s,C2H6:%s,C3H8:%s'%(h2fr,ch4fr,c2h6fr,c3h8fr), 'O2:1.0') 

print('initial parameters of mixture:')    
gas()
mol = gas.X

#calculation of frozen state after the initial shock 
U1 = sdps.CJspeed(press,temp,mol,mech,fullOutput=False)
print('Chapman - Jouguet detonation speed of the initial shock = %6.2f m/s'%U1)
gas1s = sdps.PostShock_fr(U1,press,temp,mol,mech)
print('state of gas after inital shock:')
gas1s()

#calculation of frozen state after the reflected shock
p3,UR,gas2s = sdrf.reflected_fr(gas,gas1s,gas2s,U1)
print('reflected shock speed = %6.2f m/s'%UR)
print('state of gas after reflected shock:')
gas2s()

#weld strength calculation
rad = float(input('external cylinder radius [m] = '))
wth = float(input('wall thickness [mm] = '))
wld = float(input('weld thickness (should not exceed wall thickness) [mm] = '))
apress = float(3.1415*((rad-(wth/1000))**2))
print ('area of shock [m2] = %2.4f'%apress)
aweld = float(3.1415*((rad**2)-((rad-wld/1000)**2)))
print ('area of weld [m2] = %2.4f'%aweld)

if(gas1s.P>gas2s.P):
    peff=(gas1s.P-ct.one_atm)/1e3
else:
    peff=(gas2s.P-ct.one_atm)/1e3

Feff = peff*apress
Fall = (0.6667*56.2*aweld)*1e3

print('allowable force = %5.3f kN, force after shock = %5.3f kN'%(Fall,Feff))
if(Fall>Feff):
    print('The weld is strong enough')
else:
    print('The weld is too weak')

